In [1]:
from lxml import etree
import json
import xmltodict

In [2]:
AKN = {"akn": "http://docs.oasis-open.org/legaldocml/ns/akn/3.0/CSD13"}
xml = "../data/AK-dail-2015-11-12-v2.xml"
baseURI = "http://oireachtas.ie"

In [3]:
root = etree.parse(xml)
house_name = root.xpath(".//akn:preface/akn:block[@name='proponent_ga']/akn:docProponent/text()", 
                        namespaces=AKN)
house_uri = root.find(".//{*}FRBRWork/{*}FRBRauthor").attrib['href']
date = root.find(".//{*}FRBRWork/{*}FRBRdate").attrib['date']
dbr_uri = root.xpath(".//akn:FRBRWork/akn:FRBRuri/@value", 
                                  namespaces=AKN)[0]

### Debate Headings


In [29]:
headings = {"debateList": {
            "head": { "chamber": {"showAs": house_name[0],
                                 "uri": house_uri},
                     "dateRange": {"start": date, 
                                   "end": date},
                     "query": None,
                     "by": {"showAs": None,
                            "uri": None},
                     "as": {"showAs": None,
                            "uri": None},
                     "limit": 1,
                     "skipped": 0,
                     "resource": "debateRecord",
                     "sort": {"on": "date",
                              "order": "desc"},
                     "counts": {"debateRecordCount": 1,
                               "debateSectionCount": len(root.xpath(".//akn:debateSection[./akn:heading]",
                                                                   namespaces=AKN))}
                    },
            "results": []
           }
        }

# First for loop placeholder for when it comes to parsing the xml files
for dbr in range(1):
    dbsects = root.xpath(".//akn:debateSection[./akn:heading]", namespaces=AKN)
    record = {"debateRecord": {"chamber": {"showAs": house_name[0],
                                "uri": house_uri},
                               "uri": dbr_uri, 
                               "date": date,
                               "debateSectionCount": len(dbsects), 
                               "debateSections": [] 
                               }}
    for dbs in dbsects:
        speech_count = len(dbs.xpath("./akn:speech", namespaces=AKN))
        do_children_exist = len(dbs.xpath("./akn:summary|./akn:speech|./akn:p", namespaces=AKN))>0
        parent_dbs = dbs.xpath("./parent::akn:debateSection", namespaces=AKN)
        if len(parent_dbs) > 0:
            parent_uri = dbr_uri + "/" + parent_dbs[0].attrib['eId']
            parent_heading = parent_dbs[0].find("./{*}heading").text
            parent = {"uri": parent_uri, 
                      "showAs": parent_heading}
        else:
            parent = None
        dbs_uri = dbr_uri + "/" + dbs.attrib['eId']
        heading = dbs.find("./{*}heading").text
        data = {"uri": dbs_uri, 
                "showAs": heading, 
                "context": None, 
                "speechCount": speech_count, 
                "parentDebateSection": parent,
               "containsDebate": do_children_exist}
        record['debateRecord']['debateSections'].append({"debateSection":data})
    headings['debateList']['results'].append(record)  

In [30]:
with open("../data/debate-list.json", "w") as f:
    json.dump(headings, f, sort_keys=True, indent=2)

with open("../data/debate-list.xml", "wb") as f:
    f.write(xmltodict.unparse(headings, pretty=True).encode("utf-8"))

### Parliamentary Questions

TODO: insert dot and space after question number in AKN

In [14]:
questions = {"questions": 
             {"head": { "chamber": {"showAs": house_name[0],
                                 "uri": house_uri},
                     "dateRange": {"start": date, "end": date },
                     "query": None,
                     "by": {"showAs": None,
                            "uri": None},
                     "to": {"showAs": None,
                            "uri": None},
                    "limit": 1,
                     "skipped": 0,
                    "sort": "desc",
                     "counts": {"questionCount": len(root.xpath(".//akn:question", namespaces=AKN))}
                    },
            "results": []
             }}
# if by and to are used, they are dict: {"uri": AnyURI, "showAs": string}
for q in root.xpath(".//akn:question", namespaces=AKN):
    question = {"by": {"showAs": "", 
                                  "uri": ""},
                "to": {"showAs": "", 
                                  "uri": ""},
                "debateSection": {"showAs": "", 
                                  "uri": ""},
                "uri": "",
                "context": None,
                "questionNumber": "",
               "showAs": ""}
    by_tlc = root.xpath(".//akn:TLCPerson[@eId='{}']".format(q.attrib['by'][1:]), namespaces=AKN)[0].attrib
    to_tlc = root.xpath(".//akn:TLCRole[@eId='{}']".format(q.attrib['to'][1:]), namespaces=AKN)[0].attrib
    question['uri'] = dbr_uri + "/" + q.attrib['eId']
    question['questionNumber'] = int(q.attrib['eId'].split("_")[-1])
    question['by']['uri'] = by_tlc['href']
    question['by']['showAs'] = by_tlc['showAs']
    question['to']['uri'] = to_tlc['href']
    question['to']['showAs'] = to_tlc['showAs']
    dbs = q.xpath("./parent::akn:debateSection", namespaces=AKN)[0]
    question['debateSection']['uri'] = dbr_uri + "/" + dbs.attrib['eId']
    question['debateSection']['showAs'] = dbs.xpath("./akn:heading/text()", namespaces=AKN)[0]
    question['showAs'] = " ".join(q.xpath("./akn:p//text()", namespaces=AKN)).replace("  ", " ")
    question = {"question": question}
    questions['questions']['results'].append(question)

In [15]:
with open("../data/question-list.json", "w") as f:
    json.dump(questions, f, sort_keys=True, indent=2)
    

with open("../data/question-list.xml", "wb") as f:
    f.write(xmltodict.unparse(questions, pretty=True).encode("utf-8"))

### Votes

In [32]:
questions

{'questions': {'head': {'by': {'showAs': None, 'uri': None},
   'chamber': {'showAs': 'DÁIL ÉIREANN', 'uri': '/ie/oireachtas/house/dail'},
   'counts': {'questionCount': 10},
   'dateRange': {'end': '2015-11-12', 'start': '2015-11-12'},
   'limit': 1,
   'query': None,
   'skipped': 0,
   'sort': 'desc',
   'to': {'showAs': None, 'uri': None}},
  'results': [{'question': {'by': {'showAs': 'Mr. Seán Ó Fearghaíl',
      'uri': '/ie/oireachtas/member/Seán-Ó-Fearghaíl.S.2000-06-09/dail/31'},
     'context': '1 Deputy Seán Ó Fearghaíl asked the Minister for Arts, Heritage and the Gaeltacht the funding being provided for the Heritage Council in 2016; and if she will make a statement on the matter. [39716/15]',
     'debateSection': {'showAs': 'Heritage Council Funding',
      'uri': '/akn/ie/debateRecord/dail/2015-11-12/debate/dbsect_4'},
     'questionNumber': 1,
     'to': {'showAs': 'Minister for Arts, Heritage and the Gaeltacht',
      'uri': '/ie/oireachtas/member/Heather-Humphreys.D.20